In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Model,Sequential
from keras.layers import Dense,Flatten,Dropout
from keras.optimizers import Nadam
from tensorflow.keras.metrics import Precision, Recall , F1Score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
df = pd.read_csv('/kaggle/input/list_attr_celeba.csv')
df.head()

2024-03-16 17:23:20.244120: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-16 17:23:20.244247: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-16 17:23:20.370292: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1


In [3]:
columns = df.columns

In [4]:
columns

Index(['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows',
       'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair',
       'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
       'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin',
       'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns',
       'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings',
       'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
       'Wearing_Necktie', 'Young'],
      dtype='object')

In [5]:
li = ['image_id','Attractive','Mouth_Slightly_Open','Blurry']
final = [i for i in columns if i not in li]
final

['5_o_Clock_Shadow',
 'Arched_Eyebrows',
 'Bags_Under_Eyes',
 'Bald',
 'Bangs',
 'Big_Lips',
 'Big_Nose',
 'Black_Hair',
 'Blond_Hair',
 'Brown_Hair',
 'Bushy_Eyebrows',
 'Chubby',
 'Double_Chin',
 'Eyeglasses',
 'Goatee',
 'Gray_Hair',
 'Heavy_Makeup',
 'High_Cheekbones',
 'Male',
 'Mustache',
 'Narrow_Eyes',
 'No_Beard',
 'Oval_Face',
 'Pale_Skin',
 'Pointy_Nose',
 'Receding_Hairline',
 'Rosy_Cheeks',
 'Sideburns',
 'Smiling',
 'Straight_Hair',
 'Wavy_Hair',
 'Wearing_Earrings',
 'Wearing_Hat',
 'Wearing_Lipstick',
 'Wearing_Necklace',
 'Wearing_Necktie',
 'Young']

In [6]:
df.drop(final,axis=1,inplace=True)
df.head()

,image_id,Attractive,Blurry,Mouth_Slightly_Open
0,000001.jpg,1,-1,1
1,000002.jpg,-1,-1,1
2,000003.jpg,-1,1,-1
3,000004.jpg,1,-1,-1
4,000005.jpg,1,-1,-1


In [7]:
for col in li[1:]:
    df[col] = df[col].map({-1:0,1:1})
df.head()

,image_id,Attractive,Blurry,Mouth_Slightly_Open
0,000001.jpg,1,0,1
1,000002.jpg,0,0,1
2,000003.jpg,0,1,0
3,000004.jpg,1,0,0
4,000005.jpg,1,0,0


In [8]:
path = '/kaggle/input/img_align_celeba/img_align_celeba/'
df['image_id'] = path + df['image_id']
df.head()

,image_id,Attractive,Blurry,Mouth_Slightly_Open
0,/kaggle/input/img_align_celeba/img_align_celeb...,1,0,1
1,/kaggle/input/img_align_celeba/img_align_celeb...,0,0,1
2,/kaggle/input/img_align_celeba/img_align_celeb...,0,1,0
3,/kaggle/input/img_align_celeba/img_align_celeb...,1,0,0
4,/kaggle/input/img_align_celeba/img_align_celeb...,1,0,0


In [9]:
df = df[:40000]

In [24]:
df['Attractive'].value_counts()

Attractive
1    20575
0    19425
Name: count, dtype: int64

In [25]:
df['Blurry'].value_counts()

Blurry
0    37896
1     2104
Name: count, dtype: int64

In [10]:
df['Mouth_Slightly_Open'].value_counts()

Mouth_Slightly_Open
0    20881
1    19119
Name: count, dtype: int64

In [11]:
import cv2
img = cv2.imread(df['image_id'][0])
img.shape

(218, 178, 3)

In [12]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  
)

# Create data generators for training and validation
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    x_col="image_id",  # Column containing image paths
    y_col=["Attractive", "Blurry", "Mouth_Slightly_Open"],  # Columns containing labels
    target_size=(218,178), 
    batch_size=32,
    class_mode="raw", 
    subset="training"
)

val_generator = datagen.flow_from_dataframe(
    dataframe=df,
    x_col="image_id",  # Column containing image paths
    y_col=["Attractive", "Blurry", "Mouth_Slightly_Open"],  # Columns containing labels
    target_size=(218,178),  # Resize images to fit EfficientNetB0 input shape
    batch_size=32,
    class_mode="raw",
    shuffle=False,
    subset="validation"
)

Found 32000 validated image filenames.
Found 8000 validated image filenames.


In [13]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(64, (4, 4), padding='same', input_shape=(218,178,3),activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, (4, 4), padding='same',activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(128, (4, 4), padding='same',activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(128, (4, 4), padding='same',activation='relu'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(256, (4, 4), padding='same',activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(256, (4, 4), padding='same',activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.4))

model.add(tf.keras.layers.GlobalAveragePooling2D())  # Add Global Average Pooling
model.add(tf.keras.layers.Dense(128, activation="sigmoid"))
model.add(tf.keras.layers.Dense(256, activation="sigmoid"))
# model.add(tf.keras.layers.Dense(512, activation="sigmoid"))
model.add(tf.keras.layers.Dense(3, activation='sigmoid'))
# # Compile the model
model.compile(optimizer=Nadam(), loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy',Precision(),Recall(),F1Score()])

# Reduce learning rate when a metric has stopped improving
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=2)
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath='/kaggle/working/model/attr2.keras',monitor='val_accuracy',mode='max',save_best_only=True,verbose=2)
# # Train the model
epochs = 10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data = val_generator,
    callbacks = [reduce_lr,model_checkpoint],
    batch_size=32)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


   1/1000 ━━━━━━━━━━━━━━━━━━━━ 10:12:29 37s/step - accuracy: 0.0625 - f1_score: 0.0392 - loss: 0.8575 - precision: 0.3187 - recall: 0.9667

I0000 00:00:1710610001.114705      84 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step - accuracy: 0.5833 - f1_score: 0.3689 - loss: 0.5170 - precision: 0.5755 - recall: 0.5273
Epoch 1: val_accuracy improved from -inf to 0.68800, saving model to /kaggle/working/model/attr2.keras
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 240s 204ms/step - accuracy: 0.5833 - f1_score: 0.3689 - loss: 0.5169 - precision: 0.5755 - recall: 0.5274 - val_accuracy: 0.6880 - val_f1_score: 0.4582 - val_loss: 0.5080 - val_precision: 0.7031 - val_recall: 0.3624 - learning_rate: 0.0010
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step - accuracy: 0.6319 - f1_score: 0.5941 - loss: 0.3281 - precision: 0.8150 - recall: 0.7517
Epoch 2: val_accuracy did not improve from 0.68800
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 176s 176ms/step - accuracy: 0.6319 - f1_score: 0.5941 - loss: 0.3280 - precision: 0.8150 - recall: 0.7517 - val_accuracy: 0.5116 - val_f1_score: 0.5751 - val_loss: 0.4701 - val_precision: 0.9555 - val_recall: 0.3657 - learning_rate: 0.0010
Epoch 3/10
1000/1000 ━━

In [14]:
preds = model.predict(val_generator)

250/250 ━━━━━━━━━━━━━━━━━━━━ 12s 43ms/step


In [15]:
labels = val_generator.labels
labels[0],preds[0]

(array([1, 0, 1]),
 array([9.9737877e-01, 2.3690639e-04, 5.4503185e-01], dtype=float32))

In [16]:
preds = preds>0.5

In [17]:
preds = preds.astype(int)
preds

array([[1, 0, 1],
       [1, 0, 1],
       [0, 0, 0],
       ...,
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1]])

In [18]:
from sklearn.metrics  import accuracy_score
a_true = [i[0] for i in labels]
apres = [i[0] for i in preds]
accuracy_score(a_true,apres)

0.7925

In [19]:
a_true = [i[1] for i in labels]
apres = [i[1] for i in preds]
accuracy_score(a_true,apres)

0.94475

In [20]:
a_true = [i[2] for i in labels]
apres = [i[2] for i in preds]
accuracy_score(a_true,apres)

0.9285

In [21]:
model.save("model.h5")